In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

In [2]:
import tensorflow as tf
import tf2onnx

## Reading the data

In [3]:
data = pd.read_csv(r"..\..\Data\small_ohe.csv")
data.head()

,age,default,housing,loan,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,0.171429,1,-1,1,0.029412,1.0,0.0,0,0.333333,0.269680,...,0,1,0,0,0,1,0,0,0,0
1,0.300000,1,1,1,0.088235,1.0,0.0,0,0.937500,0.698753,...,0,1,0,0,0,1,0,0,0,0
2,0.100000,1,-1,1,0.000000,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,0,0,0,0,1
3,0.285714,1,0,0,0.058824,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,1,0,0,0,0
4,0.414286,1,-1,1,0.000000,1.0,0.0,0,0.687500,0.389322,...,0,0,1,0,0,0,1,0,0,0


In [4]:
data.columns

Index(['age', 'default', 'housing', 'loan', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y', 'pdays2', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_w

In [5]:
# Saperating features and result vectors
X = data.drop('y', axis=1).values
y = data['y'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [7]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(55,),use_bias=True),
    tf.keras.layers.Dense(64, activation='relu',use_bias=True),
    tf.keras.layers.Dense(32, activation='relu',use_bias=True),
    tf.keras.layers.Dense(1, activation='sigmoid',use_bias=True)
])
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=128, epochs=60, validation_data=(X_test, y_test))

Epoch 1/60
40/40 [==============================] - 1s 4ms/step - loss: 0.6505 - accuracy: 0.6370 - val_loss: 0.5819 - val_accuracy: 0.7233
Epoch 2/60
40/40 [==============================] - 0s 2ms/step - loss: 0.5585 - accuracy: 0.7214 - val_loss: 0.5326 - val_accuracy: 0.7325
Epoch 3/60
40/40 [==============================] - 0s 2ms/step - loss: 0.5217 - accuracy: 0.7479 - val_loss: 0.5309 - val_accuracy: 0.7361
Epoch 4/60
40/40 [==============================] - 0s 2ms/step - loss: 0.5046 - accuracy: 0.7567 - val_loss: 0.4973 - val_accuracy: 0.7618
Epoch 5/60
40/40 [==============================] - 0s 2ms/step - loss: 0.4779 - accuracy: 0.7681 - val_loss: 0.4820 - val_accuracy: 0.7664
Epoch 6/60
40/40 [==============================] - 0s 1ms/step - loss: 0.4481 - accuracy: 0.7891 - val_loss: 0.4563 - val_accuracy: 0.7847
Epoch 7/60
40/40 [==============================] - 0s 1ms/step - loss: 0.4136 - accuracy: 0.8097 - val_loss: 0.4352 - val_accuracy: 0.7902
Epoch 8/60
40/40 [==

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3584      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 9,857
Trainable params: 9,857
Non-trainable params: 0
_________________________________________________________________


## Convert NN to ONNX file

In [10]:
# convert the model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(model)

# save the ONNX model to a file
with open("my_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Setup onnx

In [24]:
'''
Author: jrebort jrebort@foxmail.com
Date: 2023-03-01 00:46:04
LastEditors: jrebort jrebort@foxmail.com
LastEditTime: 2023-03-04 00:02:07
FilePath: \bank-marketing\Method\dknn\model.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
import onnx
from onnx.checker import check_model
import onnxruntime as ort

def setup(onnx_file: str,):
    # Load the ONNX model
    ort_sess = ort.InferenceSession(onnx_file)
    ortvalue = ort.OrtValue.ortvalue_from_numpy(X_train.astype(np.float32), 'cpu')
    ortvalue.device_name()  # 'cpu'
    ortvalue.shape()        # shape of the numpy array X
    ortvalue.data_type()    # 'tensor(float)'
    ortvalue.is_tensor()    # 'True'
    np.array_equal(ortvalue.numpy(), X)  # 'True'
    outputs = ort_sess.run(None, {ort_sess.get_inputs()[0]: ortvalue})
    return outputs
setup('my_model.onnx')

TypeError: run(): incompatible function arguments. The following argument types are supported:
    1. (self: onnxruntime.capi.onnxruntime_pybind11_state.InferenceSession, arg0: List[str], arg1: Dict[str, object], arg2: onnxruntime.capi.onnxruntime_pybind11_state.RunOptions) -> List[object]

Invoked with: <onnxruntime.capi.onnxruntime_pybind11_state.InferenceSession object at 0x0000022AA29C6230>, ['dense_3'], {<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x0000022AA0F62F30>: <onnxruntime.capi.onnxruntime_inference_collection.OrtValue object at 0x0000022AA2BBA790>}, None

In [49]:
ort_sess = ort.InferenceSession('my_model.onnx')
ortvalue = ort.OrtValue.ortvalue_from_numpy(X_train.astype(np.float32), 'cpu')
ortvalue.device_name()  # 'cpu'
ortvalue.shape()        # shape of the numpy array X
ortvalue.data_type()    # 'tensor(float)'
ortvalue.is_tensor()    # 'True'
np.array_equal(ortvalue.numpy(), X_train.astype(np.float32))  # 'True'
ort_sess.get_outputs()[0].name
outputs = ort_sess.run(['dense_3'], {ort_sess.get_inputs()[0].name: ortvalue})
print(outputs)


[array([[9.9999619e-01],
       [2.8908253e-06],
       [1.4162958e-03],
       ...,
       [9.9755979e-01],
       [1.8031895e-03],
       [9.9015063e-01]], dtype=float32)]


In [ ]:
def confidence(data, sess, para):
    result = np.zeros_like(data)
    return result